# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

### Некоторые вычисления для SVM
Пусть $x\in\mathbb R^n$, $y\in \{0, 1\}$

Параметры модели: $w\in\mathbb R^n$, $w_0\in\mathbb R$

Отступ: $M(x, y, w, w_0) = (2y - 1)((w, x) + w_0)$ (поскольку метки $0$, $1$ надо преобразовать к $-1$, $1$)

$\dfrac{dM}{dw} = (2y-1)x,\quad \dfrac{dM}{dw_0} = (2y-1)$

Ошибка: $L(x, y, w, w_0) = [1 - M(x, y, w, w_0)]_+ = \left\{
\begin{aligned}
& 1-M,\quad M\le1\\
& 0,\quad M > 1
\end{aligned}
\right.$

$\dfrac{dL}{dM} = \left\{
\begin{aligned}
& -1,&M < 1\\
& 0,&M > 1
\end{aligned}
\right.$

$\dfrac{dL}{dw} = \left\{
\begin{aligned}
& -(2y-1)x, & M < 1\\
& 0, & M > 1
\end{aligned}
\right.
\qquad
\dfrac{dL}{dw_0}= \left\{
\begin{aligned}
& -(2y-1),& M < 1\\
& 0,& M > 1
\end{aligned}
\right.
$

$L_2$-регуляризация

$R(w) = \dfrac{||w||^2}{2C}$

$\dfrac{dR}{dw} = \dfrac{w}{C},\quad \dfrac{dR}{dw_0} = 0$


In [2]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([self.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    # returns tuple (dR/dw, dR/dw0)
    def der_reg(self):
        return (self.w / self.C, 0.)
    
    def _margin(self, x, y):
        return (2*y - 1) * self.f(x)
    
    # hinge loss
    def loss(self, x, y):
        return max([0, 1 - _margin(x, y)])

    # hinge loss derivative
    # returns tuple (dL/dw, dL/dw0)
    def der_loss(self, x, y):
        M = self._margin(x, y)
        if M > 1:
            return (np.zeros_like(self.w), 0.)
        return (-(2*y - 1) * x, -(2*y - 1))

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(1, 10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            # step = 0.5 * 0.9 ** k
            # but it is bad so let's try other heuristic
            step = 0.1/(k ** 0.5)
            
            der_L = self.der_loss(x, y)
            der_R = self.der_reg()
            
            der = tuple(der_L[i] + der_R[i] for i in (0, 1))
            
            # w update
            self.w -= step * der[0]
            
            # w_0 update
            self.w0 -= step * der[1]

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [3]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.103633   -0.10814459 -0.02799679  0.18783359  0.12175499  0.24175446
  0.02568957  0.09778814 -0.14177277 -0.15817862  0.14030208  0.08100593
  0.24029485  0.10265193 -0.25146807  0.03528905  0.14307263  0.15529781
 -0.03183528  0.18684358] -0.13400122342716292


In [4]:
predictions = model.predict(X_test)

In [5]:
print(predictions)

[1 0 1 ... 1 1 0]


In [6]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [7]:
print(len(predictions), sum(predictions))

2000 1012


In [8]:
print(sum(predictions == y_test) / float(len(y_test)))

0.7985


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [9]:
from sklearn.svm import LinearSVC

При каждом запуске обучения получаются разные результаты, давайте обучим каждую модель несколько раз, чтобы посмотреть на качество в среднем

In [10]:
%%time

my_model = MySVM(C=10000)
my_accuracy = []
for i in range(10):
    my_model.fit(X_train, y_train)
    my_predictions = my_model.predict(X_test)
    my_accuracy.append(sum(my_predictions == y_test) / float(len(y_test)))
print("my_accuracy =", my_accuracy)
print("average =", sum(my_accuracy)/len(my_accuracy))

my_accuracy = [0.7995, 0.798, 0.7915, 0.7945, 0.799, 0.8005, 0.798, 0.801, 0.789, 0.791]
average = 0.7962
CPU times: user 2.96 s, sys: 24 ms, total: 2.98 s
Wall time: 2.96 s


In [11]:
%%time

skl_model = LinearSVC(C=10000, max_iter=10000, dual=False)
skl_accuracy = []
for i in range(10):
    skl_model.fit(X_train, y_train)
    skl_predictions = skl_model.predict(X_test)
    skl_accuracy.append(sum(skl_predictions == y_test) / float(len(y_test)))
print("skl_accuracy =", skl_accuracy)
print("average =", sum(skl_accuracy)/len(skl_accuracy))

skl_accuracy = [0.7965, 0.7965, 0.7965, 0.7965, 0.7965, 0.7965, 0.7965, 0.7965, 0.7965, 0.7965]
average = 0.7965
CPU times: user 712 ms, sys: 956 ms, total: 1.67 s
Wall time: 462 ms


## Выводы
Наша самодельная модель даёт сравнимое с моделью из sklearn качество, хотя в скорости обучения уступает.

Очевидно, что предложенный шаг $h_k = 0.5 \times 0.9^k$ является плохим выбором, поскольку начиная с некоторого $k$ шаг становится очень близок к нулю, и модель перестаёт обучаться. Например, уже при $k = 100$ шаг $h_k$ имеет порядок $10^{-5}$ и продолжает уменьшаться экспоненциально.

Были попробованы некоторые шаги вида $h_k \sim \dfrac{1}{k^\alpha}$, $\alpha \in (0, 1]$, и наиболее хорошо себя показал шаг $h_k = \dfrac{0.1}{\sqrt{k}}$ -- он дал качество такое же, как у sklearn, остальные были хуже.